# Feature Engineering + Exploración para **Producción (kg)** y Optimización del Proceso

Este notebook está pensado para:
1. **Preparar features** (tiempo, duraciones, clima, categóricas)
2. Generar **scatter plots** (variables vs producción) para identificar palancas del proceso
3. Entrenar un **baseline** rápido (opcional)
4. Exportar un dataset engineered

## Inputs
- `Dataset_Unificado.csv`
- `clima_horario_openmeteo_2025 (1).csv`


In [ ]:
# --- Imports / Setup ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Seaborn es opcional pero recomendado para LOWESS
try:
    import seaborn as sns
    sns.set(style="whitegrid")
    HAVE_SNS = True
except Exception as e:
    HAVE_SNS = False
    print("Seaborn no disponible. Instala con: pip install seaborn")
    print("Error:", e)

from pathlib import Path


## 1) Carga de datos

Ajusta rutas si mueves los archivos al repo (por ejemplo `data/`).

In [ ]:
# Rutas esperadas en un repo
PATH_UNIFICADO = Path("../data/Dataset_Unificado.csv")
PATH_CLIMA     = Path("../data/clima_horario_openmeteo_2025.csv")

# Rutas sandbox (por si estás corriendo en el entorno donde se subieron)
SANDBOX_UNIFICADO = Path("/mnt/data/Dataset_Unificado.csv")
SANDBOX_CLIMA     = Path("/mnt/data/clima_horario_openmeteo_2025 (1).csv")

def read_csv_smart(path: Path) -> pd.DataFrame:
    """Lee CSV intentando UTF-8 y luego ISO-8859-1."""
    try:
        return pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="ISO-8859-1")

if SANDBOX_UNIFICADO.exists() and SANDBOX_CLIMA.exists():
    df = read_csv_smart(SANDBOX_UNIFICADO)
    clima = read_csv_smart(SANDBOX_CLIMA)
    print("Leyendo desde /mnt/data (sandbox)")
else:
    df = read_csv_smart(PATH_UNIFICADO)
    clima = read_csv_smart(PATH_CLIMA)
    print("Leyendo desde ../data (repo)")

print("Unificado:", df.shape)
print("Clima:", clima.shape)
df.head()


## 2) Parsing de timestamps

- `Hora de inicio` (dd/mm/aaaa hh:mm)
- `time` (ISO)

In [ ]:
df = df.copy()
clima = clima.copy()

df["ts_inicio"] = pd.to_datetime(df["Hora de inicio"], dayfirst=True, errors="coerce")
clima["ts_clima"] = pd.to_datetime(clima["time"], errors="coerce")

df = df.dropna(subset=["ts_inicio"]).sort_values("ts_inicio").reset_index(drop=True)
clima = clima.dropna(subset=["ts_clima"]).sort_values("ts_clima").reset_index(drop=True)

df[["Hora de inicio", "ts_inicio"]].head()


## 3) Conversión de duraciones a numérico

- `Duración (mm:ss)` → `duracion_s`
- `Intervalo de ordeño (hh:mm)` → `intervalo_ordeño_min`

In [ ]:
def mmss_to_seconds(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if ":" not in s:
        return np.nan
    parts = s.split(":")
    if len(parts) != 2:
        return np.nan
    mm, ss = parts
    try:
        return int(mm) * 60 + int(ss)
    except ValueError:
        return np.nan

def hhmm_to_minutes(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if ":" not in s:
        return np.nan
    parts = s.split(":")
    if len(parts) != 2:
        return np.nan
    hh, mm = parts
    try:
        return int(hh) * 60 + int(mm)
    except ValueError:
        return np.nan

df["duracion_s"] = df["Duración (mm:ss)"].apply(mmss_to_seconds)
df["intervalo_ordeño_min"] = df["Intervalo de ordeño (hh:mm)"].apply(hhmm_to_minutes)

# columnas numéricas (coerce)
num_cols = [
    "Producción (kg)",
    "Duracion de Incremento",
    "Duracion de Decremento",
    "Tiempo de Incremento",
    "Tiempo de Decremento",
    "Ubre",
    "Pezón",
    "Número de ordeño",
    "Num Lactacion",
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df[[
    "Duración (mm:ss)", "duracion_s",
    "Intervalo de ordeño (hh:mm)", "intervalo_ordeño_min",
    "Producción (kg)"
]].head()


## 4) Unión con el clima (por hora)

Se une por timestamp más cercano (±2 horas) con `merge_asof`.

In [ ]:
# Renombrar columnas clima si existen con esos nombres
clima_feats = clima.rename(columns={
    "temperature_2m": "temp_C",
    "relative_humidity_2m": "hum_rel",
    "pressure_msl": "pres_msl",
    "precipitation": "precip_mm",
    "wind_speed_10m": "wind_ms",
}).copy()

# Mantener sólo lo útil si existe
keep = ["ts_clima"]
for c in ["temp_C", "hum_rel", "pres_msl", "precip_mm", "wind_ms"]:
    if c in clima_feats.columns:
        keep.append(c)

clima_feats = clima_feats[keep].sort_values("ts_clima").reset_index(drop=True)

df = pd.merge_asof(
    df.sort_values("ts_inicio"),
    clima_feats,
    left_on="ts_inicio",
    right_on="ts_clima",
    direction="nearest",
    tolerance=pd.Timedelta("2H"),
)

df[["ts_inicio"] + [c for c in ["temp_C","hum_rel","pres_msl","precip_mm","wind_ms"] if c in df.columns]].head()


## 5) Features temporales (calendario + cíclicas)

In [ ]:
ts = df["ts_inicio"]
df["hour"] = ts.dt.hour
df["dow"] = ts.dt.dayofweek
df["month"] = ts.dt.month
df["dayofyear"] = ts.dt.dayofyear
df["is_weekend"] = (df["dow"] >= 5).astype(int)

# cíclicas
df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24.0)
df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24.0)
df["doy_sin"]  = np.sin(2*np.pi*df["dayofyear"]/366.0)
df["doy_cos"]  = np.cos(2*np.pi*df["dayofyear"]/366.0)

df[["ts_inicio","hour","dow","month","is_weekend","hour_sin","hour_cos"]].head()


## 6) Features de eficiencia (útiles para optimización)

Estas no sustituyen al target, pero ayudan a diagnosticar el proceso.

In [ ]:
# Evitar divisiones por cero
df["duracion_min"] = df["duracion_s"] / 60.0
df["kg_por_min"] = np.where(df["duracion_min"] > 0, df["Producción (kg)"] / df["duracion_min"], np.nan)

# Ratios de rampas (si existen)
if "Tiempo de Incremento" in df.columns:
    df["inc_ratio"] = np.where(df["duracion_s"] > 0, df["Tiempo de Incremento"] / df["duracion_s"], np.nan)
if "Tiempo de Decremento" in df.columns:
    df["dec_ratio"] = np.where(df["duracion_s"] > 0, df["Tiempo de Decremento"] / df["duracion_s"], np.nan)

df[["Producción (kg)","duracion_s","duracion_min","kg_por_min"] + [c for c in ["inc_ratio","dec_ratio"] if c in df.columns]].head()


## 7) Codificación de categóricas (one-hot + frequency)

- One-hot para baja cardinalidad
- Frequency encoding para alta cardinalidad

In [ ]:
cat_cols = [
    "Destino Leche",
    "Tipo de evento",
    "MS",
    "Usuario",
    "Usuario.1",
    "Programa de lavado",
    "Razón de la desviación",
]
cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    df[c] = df[c].astype(str).str.strip()
    df.loc[df[c].isin(["nan", "NaN", "None", ""]), c] = np.nan

def frequency_encode(series: pd.Series) -> pd.Series:
    freq = series.value_counts(dropna=True)
    return series.map(freq).fillna(0).astype(float)

high_card, low_card = [], []
for c in cat_cols:
    nuniq = df[c].nunique(dropna=True)
    (high_card if nuniq > 30 else low_card).append(c)

for c in high_card:
    df[c + "_freq"] = frequency_encode(df[c])

df_ohe = pd.get_dummies(df[low_card], prefix=low_card, dummy_na=True) if low_card else pd.DataFrame(index=df.index)

print("One-hot (baja cardinalidad):", low_card)
print("Freq-encoding (alta cardinalidad):", high_card)
df_ohe.head()


## 8) Dataset final (features + target)

Target: **Producción (kg)**

In [ ]:
target_col = "Producción (kg)"
assert target_col in df.columns

base_feature_cols = [
    # proceso
    "duracion_s",
    "intervalo_ordeño_min",
    "Duracion de Incremento",
    "Duracion de Decremento",
    "Tiempo de Incremento",
    "Tiempo de Decremento",
    "Ubre",
    "Pezón",
    "Número de ordeño",
    "Num Lactacion",
    # tiempo
    "hour","dow","month","dayofyear","is_weekend",
    "hour_sin","hour_cos","doy_sin","doy_cos",
    # clima
    "temp_C","hum_rel","pres_msl","precip_mm","wind_ms",
    # eficiencia (diagnóstico)
    "kg_por_min",
    "inc_ratio",
    "dec_ratio",
]
base_feature_cols = [c for c in base_feature_cols if c in df.columns]

freq_cols = [c for c in df.columns if c.endswith("_freq")]

X_num = df[base_feature_cols + freq_cols].copy()
X_cat = df_ohe.copy()
X = pd.concat([X_num, X_cat], axis=1)

y = df[target_col].copy()

mask = y.notna()
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

print("X:", X.shape, "y:", y.shape)
X.head()


## 9) Scatter plots: variables vs Producción (kg)

Primero scatter simple (matplotlib). Luego, si hay seaborn, LOWESS para tendencia.

In [ ]:
# Selección de variables para scatter (ajusta libremente)
vars_scatter = [
    "duracion_s",
    "intervalo_ordeño_min",
    "Tiempo de Incremento",
    "Tiempo de Decremento",
    "Duracion de Incremento",
    "Duracion de Decremento",
    "Num Lactacion",
    "Número de ordeño",
    "temp_C",
    "hum_rel",
    "precip_mm",
    "wind_ms",
    "kg_por_min",
]

vars_scatter = [v for v in vars_scatter if v in df.columns]

# Filtrado básico: evita inf y NaN
plot_df = df.loc[mask, vars_scatter + [target_col]].replace([np.inf, -np.inf], np.nan)

print("Vars para scatter:", vars_scatter)
print("Filas disponibles (tras filtrar target):", plot_df.shape[0])


In [ ]:
# --- Scatter simple (matplotlib) ---
for v in vars_scatter:
    x = plot_df[v]
    yv = plot_df[target_col]
    m = x.notna() & yv.notna()
    if m.sum() < 10:
        continue

    plt.figure(figsize=(5,4))
    plt.scatter(x[m], yv[m], alpha=0.35, s=12)
    plt.xlabel(v)
    plt.ylabel("Producción (kg)")
    plt.title(f"Producción vs {v}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# --- Scatter + tendencia (LOWESS) si seaborn está disponible ---
if HAVE_SNS:
    for v in vars_scatter:
        x = plot_df[v]
        yv = plot_df[target_col]
        m = x.notna() & yv.notna()
        if m.sum() < 50:
            continue

        plt.figure(figsize=(5,4))
        sns.regplot(
            x=x[m],
            y=yv[m],
            scatter_kws={"alpha":0.25, "s":12},
            line_kws={"color":"red"},
            lowess=True
        )
        plt.title(f"Producción vs {v} (LOWESS)")
        plt.xlabel(v)
        plt.ylabel("Producción (kg)")
        plt.tight_layout()
        plt.show()
else:
    print("Seaborn no está disponible. Instala con: pip install seaborn")


## 10) Baseline rápido (opcional)

Sirve para validar si el set de features tiene señal.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# Quitamos columnas con demasiados NaN (simple, para baseline)
Xb = X.copy()
nan_ratio = Xb.isna().mean()
keep_cols = nan_ratio[nan_ratio <= 0.35].index.tolist()
Xb = Xb[keep_cols].fillna(Xb.median(numeric_only=True))

X_train, X_test, y_train, y_test = train_test_split(Xb, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

pred = model.predict(X_test)
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)

print("Baseline RandomForest")
print("MAE (kg):", mae)
print("R2:", r2)


## 11) Importancia de features (rápido)

Esto te ayuda a priorizar variables para optimización (palancas).

In [ ]:
importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
importances.head(20)


In [ ]:
top = importances.head(20)[::-1]
plt.figure(figsize=(7,5))
plt.barh(top.index, top.values)
plt.title("Top 20 importancias (RandomForest)")
plt.xlabel("Importancia")
plt.tight_layout()
plt.show()


## 12) Export del dataset engineered

Se exporta **X + y** para tu pipeline de ML o para reportes.

In [ ]:
out = X.copy()
out[target_col] = y

OUT_CSV = Path("/mnt/data/engineered_dataset_produccion.csv")
out.to_csv(OUT_CSV, index=False)

print("Escrito:", OUT_CSV)
out.head()
